In [1]:
from threading import Thread
import json
from stmpy import Machine, Driver
import paho.mqtt.client as mqtt
from utils import BROKER, PORT, TOPIC, QUEUE_TOPIC, HELP_TOPIC, JOIN_TOPIC
from stmpy import Driver, Machine
from threading import Thread
import paho.mqtt.client as mqtt
import ipywidgets as widgets
from IPython.display import display


class MQTT_Client_1:
    def __init__(self, teacher):
        self.teacher: Teacher = teacher
        self.count = 0
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message
        self.stm_driver: Driver = None


    def on_connect(self, client, userdata, flags, rc):
        """Called upon connecting"""
        print(f"on_connect(): {mqtt.connack_string(rc)}")


    def on_message(self, client, userdata, msg):
        """Called when receiving a message"""
        # Decode Json-message and ignore non-json formatted messages.
        try:
            message: dict = json.loads(msg.payload.decode('utf-8'))
        except json.decoder.JSONDecodeError:
            print(f"=====\nWARNING: Received message with incorrect formating:\n{msg.payload}\nIgnoring message...\n=====")
            return
        if "msg" not in message.keys():
            print(f"=====\nWARNING: Json object does not contain the key 'msg':\n{message}\nIgnoring message...\n=====")
            return
                    
        print(f"on_message(): topic: {msg.topic}, msg: {message['msg']}")

        if msg.topic == f"{TOPIC}/{JOIN_TOPIC}":
            self.setup_handler(message)
        elif msg.topic == f"{TOPIC}/{self.teacher.session_id}/{HELP_TOPIC}":
            self.help_handler(message)
        elif msg.topic == f"{TOPIC}/{self.teacher.session_id}/{QUEUE_TOPIC}":
            self.queue_handler(message)



    def validate_recipient(self, message: dict) -> bool:
        """Validate that this client is the intended recipient of this message."""
        if "ta_name" in message.keys():
            if message["ta_name"] != self.teacher.ta_name:
                print(f"{message['ta_name']} is not {self.teacher.ta_name}")
                return False
        elif "group_name" in message.keys():
            return False

        return True
        

    def subscribe_session_topics(self) -> None:
        """Subscribe to new topics after successfully joining a session"""
        self.client.unsubscribe(f"{TOPIC}/{JOIN_TOPIC}")

        self.client.subscribe(f"{TOPIC}/{self.teacher.session_id}/{QUEUE_TOPIC}")
        self.client.subscribe(f"{TOPIC}/{self.teacher.session_id}/{HELP_TOPIC}")


    def start(self, broker, port):
        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)
        self.client.subscribe(f"{TOPIC}/{JOIN_TOPIC}")

        try:
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()

    
    def setup_handler(self, message: dict):
        """Handle messages in the setup-topic"""
        # All messages in the setup topic are intended for one recipient, so validate that
        # this particular message is for us:
        if not self.validate_recipient(message):
            return
        
        print(f"Received '{message['msg']}'")
        
        # NOTE: session_created and session_joined are handled the same way.
        if message["msg"] == "session_created" or message["msg"] == "session_joined":            
            self.teacher.session_id = message["session_id"]
            self.teacher.ta_code = message["ta_code"]
            self.teacher.student_code = message["student_code"]

            self.subscribe_session_topics()

            self.stm_driver.send("session_created", "teacher")

        # Handle session_join_failed.
        elif message["msg"] == "session_join_failed":
            self.teacher.error = message["error_message"]
            self.stm_driver.send("session_join_failed", "teacher")

    
    def queue_handler(self, message: dict):
        """Handle messages in the queue-topic"""
        if message["msg"] == "queue_update":
            self.teacher.queue = message["queue"]
            if self.teacher.queue_field:
                self.teacher.queue_field.value = str(message["queue"])

    
    def help_handler(self, message: dict):
        """Handle messages in the help-topic"""
        pass
    

class Teacher:
    def __init__(self):
        # After creating/joining a session, these should be set.
        self.session_id: int = None
        self.ta_code: int = None
        self.student_code: int = None
        self.ta_name: str = None

        self.mqtt_client: mqtt.Client = None
        self.stm: Machine = None
        
        self.error = None
        self.queue: list[str] = []


    def __str__(self) -> str:
        return f"Session: {self.session_id}, TA-code: {self.ta_code}, Student-code: {self.student_code}"


    def create_session(self, b):
        """
        Create session. Send message to server and receive codes indicating the session
        has been created.
        """
        if self.ta_name_field.value == None:
            return
        
        self.ta_name = self.ta_name_field.value
        
        create_session = {"msg": "create_session", "ta_name": self.ta_name_field.value}
        self.mqtt_client.publish(f"{TOPIC}/{JOIN_TOPIC}", json.dumps(create_session, indent=4))
        
        self.stm.send("start_session")

    

    def join_session(self, b):
        """Join an already existing session using a TA code."""
        if not self.ta_code_field.value or not self.ta_name_field.value:
            return
        
        self.ta_name = self.ta_name_field.value
        
        join_session = {"msg": "join_session", "ta_code": self.ta_code_field.value, "ta_name": self.ta_name_field.value}
        self.mqtt_client.publish(f"{TOPIC}/{JOIN_TOPIC}", json.dumps(join_session, indent=4))

        self.stm.send("start_session")
        

    def in_idle(self):
        """Called upon entering idle-state."""
        print(f"In idle-state: {self}")
        self.button_create = widgets.Button(description="Create Session")
        self.button_create.on_click(self.create_session)
        self.button_join = widgets.Button(description="Join Session")
        self.button_join.on_click(self.join_session)
        self.ta_code_field = widgets.Text(value='', placeholder='', description='TA code:', disabled=False)
        self.ta_name_field = widgets.Text(value='', placeholder='', description='Name:', disabled=False)

        error_field = widgets.Text(value=self.error, placeholder='', description='Error code:', disabled=True)

        display(self.ta_name_field, self.button_create, self.ta_code_field, self.button_join, error_field)


    def in_wait(self):
        """Called upon entering wait-state."""
        print(f"In wait-state: {self}")
        return
    

    def in_lab_session_active(self):
        """Called upon entering lab_sesssion_active-state."""
        self.button_help = widgets.Button(description="Help")
        self.button_help.on_click(self.help)
        self.button_end = widgets.Button(description="End lab session")
        self.button_end.on_click(self.end_session)

        self.queue_field = widgets.Text(value=str(self.queue), placeholder="", description='Queue:', disabled=True)

        display(self.button_help, self.button_end, self.queue_field)
        

        print(f"In lab_sesssion_active-state: {self}")


    def in_help(self):
        """Called upon entering help-state."""  
        self.button_help = widgets.Button(description="Finished Helping")
        self.button_help.on_click(self.finish_help)
        display(self.button_help)

        print(f"In help-state: {self}")


    def help(self, b):
        """Called when Help-button is pressed in lab_session_active"""
        # If queue is empty, there is noone to help.
        if not self.queue:
            return

        # Inform server that group is receiving help.
        provide_help = {"msg": "provide_help", "ta_name": self.ta_name, "group_name": self.queue[0]}
        self.mqtt_client.publish(f"{TOPIC}/{self.session_id}/{HELP_TOPIC}", json.dumps(provide_help))
        
        self.stm.send("help_group")


    def finish_help(self, b):
        """Called when Finished Help-button is pressed in help"""
        # Inform server that group has been helped successfully.
        # NOTE: Probably no reason to inform the server about this, I suppose.
        # self.mqtt_client.publish(f"{TOPIC}/{self.session_id}/{HELP_TOPIC}", "finished helping")
        self.stm.send("finish_help")


    def end_session(self, b):
        """Called when End Session-button is pressed in lab_session_active"""
        self.stm.send("end_lab")


In [2]:
# initial transition
t0 = {'source': 'initial',
      'target': 'idle'}

# transitions
t1 = {'trigger':'start_session', 
      'source':'idle', 
      'target':'wait'}
t2 = {'trigger':'session_created', 
      'source':'wait', 
      'target':'lab_session_active'}
t3 = {'trigger':'session_join_failed', 
      'source':'wait', 
      'target':'idle'}

t4 = {'trigger':'help_group', 
      'source':'lab_session_active', 
      'target':'help'}
t5 = {'trigger':'finish_help', 
      'source':'help', 
      'target':'lab_session_active'}

t6 = {'trigger':'task_done', 
      'source':'lab_session_active', 
      'target':'lab_session_active'}
t7 = {'trigger':'update_queue', 
      'source':'lab_session_active', 
      'target':'lab_session_active'}

t8 = {'trigger':'end_lab', 
      'source':'lab_session_active', 
      'target':'exit'}
 
# the states:
idle = {'name': 'idle',
        'entry': 'in_idle'}

wait = {'name': 'wait',
        'entry': 'in_wait'}

lab_session_active = {'name': 'lab_session_active',
        'entry': 'in_lab_session_active'}

help = {'name': 'help',
        'entry': 'in_help'}

In [ ]:
teacher = Teacher()
state_machine = Machine(transitions=[t0, t1, t2, t3, t4, t5, t6, t7, t8], states=[idle, wait, lab_session_active, help], obj=teacher, name="teacher")
teacher.stm = state_machine

driver = Driver()
driver.add_machine(state_machine)

myclient = MQTT_Client_1(teacher)
teacher.mqtt_client = myclient.client
myclient.stm_driver = driver

driver.start()
myclient.start(BROKER, PORT)

In idle-state: Session: None, TA-code: None, Student-code: None
Connecting to mqtt20.iik.ntnu.no:1883


Text(value='', description='Name:', placeholder='')

Button(description='Create Session', style=ButtonStyle())

Text(value='', description='TA code:', placeholder='')

Button(description='Join Session', style=ButtonStyle())

...

on_message(): topic: team02_testing/0/help, msg: provide_help
In help-state: Session: 0, TA-code: 388, Student-code: 20928
on_message(): topic: team02_testing/0/queue, msg: queue_update


Button(description='Help', style=ButtonStyle())

Button(description='End lab session', style=ButtonStyle())

Text(value='[]', description='Queue:', disabled=True, placeholder='')

In lab_sesssion_active-state: Session: 0, TA-code: 388, Student-code: 20928


In [4]:
# TODO: Would be nice to be able to run two clients here at the same time. For some 
# reason they are interfering with eachother when set up like this.

# teacher2 = Teacher()
# state_machine2 = Machine(transitions=[t0, t1, t2, t3, t4, t5, t6, t7, t8], states=[idle, wait, lab_session_active, help], obj=teacher2, name="teacher")
# teacher2.stm = state_machine2

# driver2 = Driver()
# driver2.add_machine(state_machine2)

# myclient2 = MQTT_Client_1(teacher2)
# teacher2.mqtt_client = myclient2.client
# myclient2.stm_driver = driver2

# driver2.start()
# myclient2.start(BROKER, PORT)